In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

In [2]:
env = UnityEnvironment(file_name='b6-uibuild\SoccerBot')




In [3]:
env.reset()

In [4]:
behavior_names = list(env.behavior_specs.keys())
print(behavior_names)
action_tuple = env.behavior_specs[behavior_names[0]][1].empty_action(1)

['Soccer Bot?team=0', 'Soccer Bot?team=1']


In [5]:
filepath_ppo_t0 = 'team_0/PPO'
filepath_a2c_t0 = 'team_0/A2C'
model_no_a2c = 999
model_no_ppo = 951

filepath_a2c_t1 = 'team_1'
filepath_ppo_t1 = 'team_1'

In [6]:
model_t0 = PPO.load(f'{filepath_ppo_t0}/PPO_soccerbot_t0_{model_no_ppo}.zip')
model_t1 = PPO.load(f'{filepath_ppo_t1}/ppo_soccerbot_t1.zip')

model_t1.set_parameters(model_t0.get_parameters())

In [7]:
# model_t0 = A2C.load(f'{filepath_a2c_t0}/a2c_soccerbot_t0_{model_no_a2c}.zip')
# model_t1 = A2C.load(f'{filepath_a2c_t1}/a2c_soccerbot_t1.zip')

# model_t1.set_parameters(model_t0.get_parameters())

In [8]:
def getobservation( decstep , termstep ):
    if decstep.obs[0].shape == (2,56) and decstep.obs[1].shape == (2,168) :
        obs = np.concatenate((decstep.obs[0],decstep.obs[1]) , axis=1 , dtype=np.float32)            
    else:
        obs = np.concatenate((termstep.obs[0],termstep.obs[1]) , axis=1, dtype=np.float32)
    obs = np.reshape(obs ,newshape=(448,))
    return obs

In [9]:
def setaction_t0(model , obs , decstep , behavior_name ):
    global model_t0 
    act_t1,_ = model.predict(observation=obs , deterministic=True)
        
    idx: int = 0
    for agent in decstep.agent_id:
        action_tuple.add_discrete(np.array(act_t1[2*idx : 2*(idx+1)]).reshape((1,2)))
        env.set_action_for_agent(behavior_name,agent,action=action_tuple)
        idx+=1

def setaction_t1(model , obs , decstep , behavior_name ):
    global model_t1
    act_t1,_ = model.predict(observation=obs , deterministic=True)
        
    idx: int = 0
    for agent in decstep.agent_id:
        action_tuple.add_discrete(np.array(act_t1[2*idx : 2*(idx+1)]).reshape((1,2)))
        env.set_action_for_agent(behavior_name,agent,action=action_tuple)
        idx+=1

In [10]:

n_steps = 2500
for step in range(n_steps):
    (decstept0 , termstept0) = env.get_steps(behavior_name=behavior_names[0])
    (decstept1 , termstept1) = env.get_steps(behavior_name=behavior_names[1])
    
    obs = getobservation(decstept0 , termstept0)
    setaction_t0(model_t0 , obs , decstept0 , behavior_names[0])
    
    obs = getobservation(decstept1 , termstept1)
    setaction_t1(model_t1 , obs , decstept1 , behavior_names[1])

    env.step()

env.close()

UnityCommunicatorStoppedException: Communicator has exited.